In [23]:
import torch
import numpy as np

## Using normal autodifferentiation approach

In [46]:
learning_rate = 1e-2

q0 = torch.tensor([1,0], dtype=torch.complex128, requires_grad=False)

α = torch.rand(1, dtype=torch.float64, requires_grad = True)
β = torch.rand(1, dtype=torch.float64, requires_grad = True)
θ = torch.rand(1, dtype=torch.float64, requires_grad = True)
ϕ = torch.rand(1, dtype=torch.float64, requires_grad = True)

# U = torch.rand(2,2, dtype=torch.complex128, requires_grad=True)
H = torch.tensor([[1,1],[1,-1]] / np.sqrt(2), dtype=torch.complex128, requires_grad=False)

def elements_to_matrix(matrix_entries: list):
    return torch.stack([torch.stack([value for value in row]) for row in matrix_entries]).squeeze()
    
def quantum_infidelity(state, target_state):
    return 1 - torch.dot(state.conj(), target_state)**2

optimizer = torch.optim.Adam([α, β, θ, ϕ], lr=learning_rate)

print(elements_to_matrix([[torch.tensor(1), torch.tensor(2)],[torch.tensor(3), torch.tensor(4)]]))

for epoch in range(1000):
    U = torch.exp(1j * ϕ / 2) * elements_to_matrix([
        [torch.exp(1j * α) * torch.cos(θ), torch.exp(1j * β) * torch.sin(θ)],
        [- torch.exp(-1j * β) * torch.sin(θ), torch.exp(-1j * α) * torch.cos(θ)]
    ])
    q0_out = U.matmul(q0)
    q0_target = H.matmul(q0)
    loss = quantum_infidelity(q0_out, q0_target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 200 == 0:
        print(f"U:    {U}")
        print(f"Loss: {loss}")
        print()

tensor([[1, 2],
        [3, 4]])
U:    tensor([[ 0.6070+0.1734j,  0.5411+0.5555j],
        [-0.5613+0.5351j,  0.6131-0.1508j]], dtype=torch.complex128,
       grad_fn=<MulBackward0>)
Loss: (1.2499594354453016+0.032396773404116j)

U:    tensor([[ 0.7040-0.0059j, -0.6919-0.1599j],
        [ 0.7101+0.0059j,  0.6833+0.1699j]], dtype=torch.complex128,
       grad_fn=<MulBackward0>)
Loss: (8.761761998399287e-05-5.304814132425017e-06j)

U:    tensor([[ 0.7071+1.5332e-07j, -0.6875-1.6531e-01j],
        [ 0.7071-1.5506e-07j,  0.6875+1.6531e-01j]], dtype=torch.complex128,
       grad_fn=<MulBackward0>)
Loss: (1.1368683772161603e-13-2.4665192218120803e-09j)

U:    tensor([[ 0.7071+4.1778e-12j, -0.6875-1.6531e-01j],
        [ 0.7071-3.8792e-12j,  0.6875+1.6531e-01j]], dtype=torch.complex128,
       grad_fn=<MulBackward0>)
Loss: (4.440892098500626e-16+4.223744975416693e-13j)

U:    tensor([[ 0.7071-8.3267e-17j, -0.6875-1.6531e-01j],
        [ 0.7071+6.9389e-17j,  0.6875+1.6531e-01j]], dtype=torch.c

In [47]:
with torch.no_grad():
    print(q0)
    print(U.matmul(q0))

tensor([1.+0.j, 0.+0.j], dtype=torch.complex128)
tensor([0.7071+0.0000e+00j, 0.7071-1.3878e-17j], dtype=torch.complex128)


## Using `torch.nn.Module`

In [116]:
class SingleQubitGate(torch.nn.Module):
    
    def __init__(self):
        super().__init__()
        self.α = torch.nn.Parameter(torch.rand(1, dtype=torch.float64, requires_grad = True))
        self.β = torch.nn.Parameter(torch.rand(1, dtype=torch.float64, requires_grad = True))
        self.θ = torch.nn.Parameter(torch.rand(1, dtype=torch.float64, requires_grad = True))
        self.ϕ = torch.nn.Parameter(torch.rand(1, dtype=torch.float64, requires_grad = True))
        
    def forward(self, x):
        if len(x.shape) == 1: # a single vector
            return self._forward_single(x)
        else:
            return self._forward_batch(x)
        
    def _forward_single(self, x):
        α, β, θ, ϕ = self.α, self.β, self.θ, self.ϕ
        U = torch.exp(1j * ϕ / 2) * elements_to_matrix([
            [torch.exp(1j * α) * torch.cos(θ), torch.exp(1j * β) * torch.sin(θ)],
            [- torch.exp(-1j * β) * torch.sin(θ), torch.exp(-1j * α) * torch.cos(θ)]
        ])
        return U.matmul(x)
    
    def _forward_batch(self, x_batch):
        α, β, θ, ϕ = self.α, self.β, self.θ, self.ϕ
        U = torch.exp(1j * ϕ / 2) * elements_to_matrix([
            [torch.exp(1j * α) * torch.cos(θ), torch.exp(1j * β) * torch.sin(θ)],
            [- torch.exp(-1j * β) * torch.sin(θ), torch.exp(-1j * α) * torch.cos(θ)]
        ])
        return torch.einsum('ij,bj->bi', U, x_batch)

### Train on single example

In [117]:
model = SingleQubitGate()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(1000):
    q0_out = model.forward(q0)
    q0_target = H.matmul(q0)
    loss = quantum_infidelity(q0_out, q0_target)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    if epoch % 200 == 0:
        print(f"Loss: {loss}")
        
with torch.no_grad():
    print(q0)
    print(model.forward(q0))

Loss: (1.0583467340989674+0.23948364300654784j)
Loss: (1.1296647650649305e-07-7.164701218646363e-06j)
Loss: (4.440892098500626e-16+8.671024406699302e-10j)
Loss: (4.440892098500626e-16-3.41495109763721e-14j)
Loss: (2.220446049250313e-16+9.813077866773592e-18j)
tensor([1.+0.j, 0.+0.j], dtype=torch.complex128)
tensor([0.7071-6.9389e-18j, 0.7071+1.3878e-17j], dtype=torch.complex128)


### Using dataloader

In [118]:
import pandas as pd
df = pd.read_csv('qubit.csv')

In [119]:
batch_size = 8

class QubitDataset(torch.utils.data.Dataset):
    def __init__(self):
        
        self.df = pd.read_csv('qubit.csv')
        self.df['input1'] = self.df['input1'].astype(complex)
        self.df['input2'] = self.df['input2'].astype(complex)
        self.df['output1'] = self.df['output1'].astype(complex)
        self.df['output2'] = self.df['output2'].astype(complex)

        dataset = []
        labels = []

        for i in range(batch_size):
            dataset.append([self.df['input1'][i],self.df['input2'][i]])
            
        self.dataset = torch.tensor(dataset, dtype=torch.complex128)
        
        for i in range(batch_size):
            labels.append([self.df['output1'][i],self.df['output2'][i]])
        
        self.labels = torch.tensor(labels, dtype=torch.complex128)
        
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        return self.dataset[idx],self.labels[idx]

data_set = QubitDataset()
data_loader = torch.utils.data.DataLoader(data_set, batch_size=batch_size, shuffle=False, drop_last=False)

In [120]:
model = SingleQubitGate()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

def quantum_infidelity_batched(state_batch, target_state_batch):
    return torch.stack([torch.abs(1 - torch.abs(torch.dot(state.conj(), target_state))**2)
                        for state, target_state in zip(state_batch, target_state_batch)]).mean()

for epoch in range(1000):
    for batch, labels in data_loader:
        # print(batch)
        # print(labels)
        outputs = model(batch)
        loss = quantum_infidelity_batched(outputs, labels) 
        loss.backward()
        optimizer.step()
    if epoch % 200 == 0:
        print(f"Loss: {loss}")
        # print(outputs)
        # print(labels)
        
with torch.no_grad():
    print(q0)
    print(model.forward(q0))

Loss: 0.6161921476104731
Loss: 0.43762730818361684
Loss: 0.5680465862271575
Loss: 0.385148277416396
Loss: 0.37380097606240853
tensor([1.+0.j, 0.+0.j], dtype=torch.complex128)
tensor([-0.0430-0.0056j,  0.9884+0.1454j], dtype=torch.complex128)
